In [1]:
import pandas as pd
import unicodedata
import re
from difflib import get_close_matches

In [2]:
stats_2025=pd.read_csv('../stats/batter_stats_2025.csv', encoding='utf-8', 
            encoding_errors='replace')  # replaces invalid bytes with �


In [3]:
stats_2024 = pd.read_csv('../stats/batter_stats_2024.csv')
# stats_2025 = pd.read_csv('../stats/batter_stats_2025.csv')
def_rank_2024 = pd.read_csv('def_rank_hits_allowed_pitchers_2024.csv')
def_rank_2025 = pd.read_csv('../../scrapers/def_scraper/batter_hits_def_rank.csv')
batter_team_2024 = pd.read_csv('../stats/batters_team_2024.csv')
batter_team_2025 = pd.read_csv('../stats/batters_team_2025.csv')

In [4]:
#dropping first half of the 2024 season
stats_2024 = stats_2024.iloc[len(stats_2024)//2:]

In [5]:
def_rank_2025.rename(columns={'Team': 'team'}, inplace=True)


In [6]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'OAK',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank_2024['team'] = def_rank_2024['team'].map(team_mapping)
def_rank_2025['team'] = def_rank_2025['team'].map(team_mapping)

In [7]:
#fixing batter_team_2024
batter_team_2024['team'] = batter_team_2024['team'].replace({'WSN': 'WSH'})
batter_team_2025['team'] = batter_team_2025['team'].replace({'WSN': 'WSH'})

In [8]:
stats_2024=stats_2024[['away_hitters','away_H','home_hitters','home_H']]
stats_2025=stats_2025[['away_hitters','away_H','home_hitters','home_H']]

In [9]:
#normalizing the names, getting rid of accents
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [10]:
batter_team_2024['player'] = batter_team_2024['player'].apply(normalize_name)
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [11]:
#getting rid of the # and the * from the names
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')



In [12]:
#applying it to the player collumn
batter_team_2024['player'] = batter_team_2024['player'].apply(clean_name_symbols)
batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)

In [ ]:
multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
filtered_df = batter_team_2024[~batter_team_2024['team'].isin(multi_team_flags)].copy()
filtered_df_2 = batter_team_2025[~batter_team_2025['team'].isin(multi_team_flags)].copy()


# Step 2: Keep only the LAST team listed for each player (i.e., current team)
batter_team_2024 = filtered_df.drop_duplicates(subset='player', keep='last').reset_index(drop=True)
batter_team_2025 = filtered_df_2.drop_duplicates(subset='player', keep='last').reset_index(drop=True)

In [15]:
#matching the batter_team_2024 with the broken names in stats_2024 
# Step 1: Safely extract short names like "P. Sandoval"
def clean_name(raw):
    if not isinstance(raw, str):
        return None
    match = re.match(r"([A-Z]\.\s[A-Za-z'-]+)", raw)
    return match.group(1) if match else None

stats_2024['short_name'] = stats_2024['away_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2024['player'].tolist()
stats_2024['matched_name'] = stats_2024['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df = stats_2024.merge(batter_team_2024, left_on='matched_name', right_on='player', how='left')


In [16]:
#dropping the collumns we used to match to then match the other column
merged_df = merged_df.drop(columns=['short_name', 'matched_name'])

In [17]:
#matching the batter_team_2025 with the broken names in stats_2025 
# Step 1: Safely extract short names like "P. Sandoval"
def clean_name(raw):
    if not isinstance(raw, str):
        return None
    match = re.match(r"([A-Z]\.\s[A-Za-z'-]+)", raw)
    return match.group(1) if match else None

stats_2025['short_name'] = stats_2025['away_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2025['player'].tolist()
stats_2025['matched_name'] = stats_2025['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_5 = stats_2025.merge(batter_team_2025, left_on='matched_name', right_on='player', how='left')


In [18]:
#dropping the collumns we used to match to then match the other column
merged_df_5 = merged_df_5.drop(columns=['short_name', 'matched_name'])


In [19]:
################# THIS IS WHERE I STOPPED FOR THE CLOUD

In [20]:
#now we are doing the same process except on the home players this time
# Step 1: Safely extract short names like "P. Sandoval"
def clean_name(raw):
    if not isinstance(raw, str):
        return None
    match = re.match(r"([A-Z]\.\s[A-Za-z'-]+)", raw)
    return match.group(1) if match else None

merged_df['short_name'] = merged_df['home_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2024['player'].tolist()
merged_df['matched_name'] = merged_df['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_2 = merged_df.merge(batter_team_2024, left_on='matched_name', right_on='player', how='left')

In [21]:
#now we are doing the same process except on the home players this time
# Step 1: Safely extract short names like "P. Sandoval"
def clean_name(raw):
    if not isinstance(raw, str):
        return None
    match = re.match(r"([A-Z]\.\s[A-Za-z'-]+)", raw)
    return match.group(1) if match else None

merged_df_5['short_name'] = merged_df_5['home_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2025['player'].tolist()
merged_df_5['matched_name'] = merged_df_5['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_6 = merged_df_5.merge(batter_team_2025, left_on='matched_name', right_on='player', how='left')

In [22]:
#dropping a lot of uneeded columns for simplicity
merged_df_2=merged_df_2[['player_x','team_x','away_H','player_y','team_y','home_H']]


In [23]:
merged_df_7=merged_df_6[['player_x','team_x','away_H','player_y','team_y','home_H']]

In [24]:
# Rename the column
def_rank_2024.rename(columns={'Rank': 'def_rank'}, inplace=True)
def_rank_2025.rename(columns={'Rank': 'def_rank'}, inplace=True)

In [25]:
#making both types strings
merged_df_2['team_x'] = merged_df_2['team_x'].astype(str)
def_rank_2024['def_rank'] = def_rank_2024['def_rank'].astype(str)
def_rank_2025['def_rank'] = def_rank_2025['def_rank'].astype(str)
merged_df_7['team_x'] = merged_df_7['team_x'].astype(str)


C:\Users\Joshua\AppData\Local\Temp\ipykernel_126720\394678676.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_7['team_x'] = merged_df_7['team_x'].astype(str)


In [26]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_2 = merged_df_2.merge(
    def_rank_2024,
    how='left',
    left_on='team_x',
    right_on='team'
)
merged_df_2 = merged_df_2.drop(columns=['team'])


In [27]:
#merge df_rank_2025 into merged_df_7 to bring in the def_rank 
merged_df_7 = merged_df_7.merge(
    def_rank_2025,
    how='left',
    left_on='team_x',
    right_on='team'
)
merged_df_7 = merged_df_7.drop(columns=['team'])

In [28]:
#merging for home players def rank
merged_df_3 = merged_df_2.merge(
    def_rank_2024,
    how='left',
    left_on='team_y',
    right_on='team'
)
merged_df_3 = merged_df_3.drop(columns=['team'])

In [29]:
#merging for home players def rank
merged_df_8 = merged_df_7.merge(
    def_rank_2025,
    how='left',
    left_on='team_y',
    right_on='team'
)
merged_df_8 = merged_df_8.drop(columns=['team'])

In [30]:
#dropping uneeded created collumns from merging and making new df so we arent using slices of a df
df=merged_df_3[['player_x','team_x','away_H','def_rank_y']]
df2=merged_df_3[['player_y','team_y','home_H','def_rank_x']]
#2025
df3=merged_df_8[['player_x','team_x','away_H','def_rank_y']]
df4=merged_df_8[['player_y','team_y','home_H','def_rank_x']]

In [31]:
#dropping all the nan corows
df.dropna()
df2.dropna()
df3.dropna()
df4.dropna()

,player_y,team_y,home_H,def_rank_x
0,Austin Wells,NYY,1.0,16
1,Aaron Judge,NYY,1.0,16
2,Cody Bellinger,NYY,1.0,13
3,Paul Goldschmidt,NYY,0.0,16
4,Jazz Chisholm Jr.,NYY,0.0,16
...,...,...,...,...
4917,Jose Ramirez,CLE,1.0,19
4918,Carlos Santana,CLE,0.0,19
4919,Kyle Manzardo,CLE,0.0,19
4920,Bo Naylor,CLE,0.0,19


In [32]:
#renaming the columns of the away players 
df = df.rename(columns={
    'player_x': 'player',
    'team_x': 'team',
    'away_H': 'hits',
    'def_rank_y': 'def_rank'
})

In [33]:
#renaming the columns of the away players 
df3 = df3.rename(columns={
    'player_x': 'player',
    'team_x': 'team',
    'away_H': 'hits',
    'def_rank_y': 'def_rank'
})

In [34]:
#renaming the columns of the home players 

df2 = df2.rename(columns={
    'player_y': 'player',
    'team_y': 'team',
    'home_H': 'hits',
    'def_rank_x': 'def_rank'
})


In [35]:
#renaming the columns of the home players 

df4 = df4.rename(columns={
    'player_y': 'player',
    'team_y': 'team',
    'home_H': 'hits',
    'def_rank_x': 'def_rank'
})


In [36]:
#converting types to do math on it
# Convert 'hits' to numeric (errors='coerce' turns bad values into NaN)
df['hits'] = pd.to_numeric(df['hits'], errors='coerce')
df2['hits'] = pd.to_numeric(df2['hits'], errors='coerce')
df3['hits'] = pd.to_numeric(df3['hits'], errors='coerce')
df4['hits'] = pd.to_numeric(df4['hits'], errors='coerce')


In [37]:
#putting 1s in to indicate if they got 1 or 2 hits based off the column
df['1_hit_count'] = (df['hits'] > 0).astype(int)
df['2_hit_count'] = (df['hits'] > 1.5).astype(int)
df2['1_hit_count'] = (df2['hits'] > 0).astype(int)
df2['2_hit_count'] = (df2['hits'] > 1.5).astype(int)
#2025
df3['1_hit_count'] = (df3['hits'] > 0).astype(int)
df3['2_hit_count'] = (df3['hits'] > 1.5).astype(int)
df4['1_hit_count'] = (df4['hits'] > 0).astype(int)
df4['2_hit_count'] = (df4['hits'] > 1.5).astype(int)

In [38]:
#dropping nan collumns
df.dropna()
df2.dropna()
df3.dropna()
df4.dropna()

,player,team,hits,def_rank,1_hit_count,2_hit_count
0,Austin Wells,NYY,1.0,16,1,0
1,Aaron Judge,NYY,1.0,16,1,0
2,Cody Bellinger,NYY,1.0,13,1,0
3,Paul Goldschmidt,NYY,0.0,16,0,0
4,Jazz Chisholm Jr.,NYY,0.0,16,0,0
...,...,...,...,...,...,...
4917,Jose Ramirez,CLE,1.0,19,1,0
4918,Carlos Santana,CLE,0.0,19,0,0
4919,Kyle Manzardo,CLE,0.0,19,0,0
4920,Bo Naylor,CLE,0.0,19,0,0


In [39]:
#merging one row from df and then the next row from df2 and back and fourth
# Make sure both DataFrames have the same columns
df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
# Stack them row by row
interleaved = pd.concat([df, df2]).sort_index(kind='merge').reset_index(drop=True)


In [40]:
interleaved_2 = pd.concat([df3, df4]).sort_index(kind='merge').reset_index(drop=True)


In [41]:
#dopping nans
interleaved.dropna()
interleaved_2.dropna()

,player,team,hits,def_rank,1_hit_count,2_hit_count
0,Jackson Chourio,MIL,0.0,1,0,0
1,Austin Wells,NYY,1.0,16,1,0
2,Christian Yelich,MIL,0.0,1,0,0
3,Aaron Judge,NYY,1.0,16,1,0
4,Willson Contreras,STL,0.0,1,0,0
...,...,...,...,...,...,...
9839,Kyle Manzardo,CLE,0.0,19,0,0
9840,Max Kepler,PHI,0.0,24,0,0
9841,Bo Naylor,CLE,0.0,19,0,0
9844,Alec Bohm,PHI,2.0,24,1,1


In [42]:
#making def_rank column numeric to use to split data
interleaved['def_rank'] = pd.to_numeric(interleaved['def_rank'], errors='coerce')
interleaved_2['def_rank'] = pd.to_numeric(interleaved_2['def_rank'], errors='coerce')


In [43]:
#unders are only against top 15 teams and overs are only against bot 15 teams, so we split then to get the overs and unders df
unders = interleaved[interleaved['def_rank'] < 16].reset_index(drop=True)
overs = interleaved[interleaved['def_rank'] > 15].reset_index(drop=True)
#2025
unders_2025 = interleaved_2[interleaved_2['def_rank'] < 16].reset_index(drop=True)
overs_2025 = interleaved_2[interleaved_2['def_rank'] > 15].reset_index(drop=True)



In [44]:
#dropping nans
unders.dropna()
overs.dropna()
unders_2025.dropna()
overs_2025.dropna()

,player,team,hits,def_rank,1_hit_count,2_hit_count
0,Austin Wells,NYY,1.0,16.0,1,0
1,Aaron Judge,NYY,1.0,16.0,1,0
2,Paul Goldschmidt,NYY,0.0,16.0,0,0
3,Jazz Chisholm Jr.,NYY,0.0,16.0,0,0
4,Jasson Dominguez,NYY,0.0,16.0,0,0
...,...,...,...,...,...,...
3329,Kyle Manzardo,CLE,0.0,19.0,0,0
3330,Max Kepler,PHI,0.0,24.0,0,0
3331,Bo Naylor,CLE,0.0,19.0,0,0
3333,Alec Bohm,PHI,2.0,24.0,1,1


In [45]:
#stacking overs on top of overs_2025 to make sure they are in order to count freq
stacked_overs = pd.concat([overs, overs_2025], ignore_index=True)

In [46]:
#stacking unders on top of unders_2025 to make sure they are in order to count freq
stacked_unders = pd.concat([unders, unders_2025], ignore_index=True)

In [47]:
#making games collumn
stacked_overs['games_played'] = 1
stacked_unders['games_played'] = 1

In [48]:
#################### THIS IS WHERE TO ADD THE LIMITATION OF DATA ON THE PREV YEAR AKA DROP THE BEGINING HALF OF 2024 DATA

In [49]:
stacked_overs.dropna()
stacked_unders.dropna()


,player,team,hits,def_rank,1_hit_count,2_hit_count,games_played
0,Andrew McCutchen,PIT,0.0,8.0,0,0,1
1,Bryan Reynolds,PIT,1.0,8.0,1,0,1
2,Connor Joe,PIT,1.0,8.0,1,0,1
4,Edward Olivares,PIT,1.0,8.0,1,0,1
5,Nick Gonzales,PIT,0.0,8.0,0,0,1
...,...,...,...,...,...,...,...
13272,Willi Castro,MIN,0.0,6.0,0,0,1
13273,Randy Arozarena,SEA,1.0,15.0,1,0,1
13276,Mitch Garver,SEA,0.0,15.0,0,0,1
13277,Myles Straw,TOR,1.0,6.0,1,0,1


In [50]:
#now i have to stack the years on top of eachother


In [51]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for overs
overs_freq_2 = stacked_overs.groupby('player', as_index=False).agg({
    '1_hit_count': 'sum',
    '2_hit_count': 'sum',
    'games_played': 'sum'
})

# Add a column for hit rate aka freq
overs_freq_2['1_hit_rate'] = overs_freq_2['1_hit_count'] / overs_freq_2['games_played']
overs_freq_2['2_hit_rate'] = overs_freq_2['2_hit_count'] / overs_freq_2['games_played']


In [52]:
#dropping all players that do not have at least 10 games played in this category
overs_freq_2 = overs_freq_2[overs_freq_2['games_played'] > 14].reset_index(drop=True)


In [53]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for unders 
unders_freq_2 = stacked_unders.groupby('player', as_index=False).agg({
    '1_hit_count': 'sum',
    '2_hit_count': 'sum',
    'games_played': 'sum'
})

# Add a column for hit rate
unders_freq_2['1_hit_rate'] = unders_freq_2['1_hit_count'] / unders_freq_2['games_played']
unders_freq_2['2_hit_rate'] = unders_freq_2['2_hit_count'] / unders_freq_2['games_played']

In [54]:
#dropping all players that do not have at least 10 games played in this category
unders_freq_2 = unders_freq_2[unders_freq_2['games_played'] > 14].reset_index(drop=True)


In [55]:
#checking if distrobition is correct, merging them on name to make sure they average less hits against better pitchers and vise versa
final_freq4 = overs_freq_2.merge(
    unders_freq_2,
    on='player',
    how='inner',
    suffixes=('_over', '_under')
)

In [56]:
#dropping all rows that do not meet the creteria: avg less hits against better teams, and more against worse teams
final_freq4 = final_freq4[final_freq4['1_hit_rate_under'] <= final_freq4['1_hit_rate_over']].reset_index(drop=True)

In [57]:
#making the final dataframes before sending them to the final script with odds
final_over_df=final_freq4[['player','1_hit_rate_over','2_hit_rate_over']]
final_under_df=final_freq4[['player','1_hit_rate_under','2_hit_rate_under']]


In [58]:
final_over_df

,player,1_hit_rate_over,2_hit_rate_over
0,Aaron Judge,0.842105,0.421053
1,Adam Frazier,0.478261,0.086957
2,Addison Barger,0.625000,0.125000
3,Adley Rutschman,0.615385,0.102564
4,Adolis Garcia,0.702128,0.212766
...,...,...,...
195,Wyatt Langford,0.636364,0.303030
196,Yandy Diaz,0.708333,0.375000
197,Yordan Alvarez,0.777778,0.400000
198,Zach McKinstry,0.578947,0.210526


In [59]:
final_under_df

,player,1_hit_rate_under,2_hit_rate_under
0,Aaron Judge,0.743590,0.384615
1,Adam Frazier,0.285714,0.142857
2,Addison Barger,0.466667,0.133333
3,Adley Rutschman,0.594595,0.189189
4,Adolis Garcia,0.518519,0.148148
...,...,...,...
195,Wyatt Langford,0.514286,0.114286
196,Yandy Diaz,0.689189,0.270270
197,Yordan Alvarez,0.636364,0.212121
198,Zach McKinstry,0.473684,0.236842


In [60]:
######## EXPLANING: AARON JUDGE GETS A HIT 74.% OF THE TIME VS BOT 15 TEAMS AND 2 HITS 42% OF THE TIME VS BOT 15 TEAMS
final_over_df.to_csv('over_hit_rate.csv')

In [61]:
######## EXPLANING: AARON JUDGE GETS A HIT 71.87% OF THE TIME VS TOP 15 TEAMS AND 2 HITS 29% OF THE TIME VS TOP 15 TEAMS
final_under_df.to_csv('under_hit_rate.csv')